In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("free_throws.csv")

In [2]:
data.head()

,end_result,game,game_id,period,play,player,playoffs,score,season,shot_made,time
0,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 1 of 2,Andrew Bynum,regular,0 - 1,2006 - 2007,1,11:45
1,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 2 of 2,Andrew Bynum,regular,0 - 2,2006 - 2007,1,11:45
2,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum makes free throw 1 of 2,Andrew Bynum,regular,18 - 12,2006 - 2007,1,7:26
3,106 - 114,PHX - LAL,261031013.0,1.0,Andrew Bynum misses free throw 2 of 2,Andrew Bynum,regular,18 - 12,2006 - 2007,0,7:26
4,106 - 114,PHX - LAL,261031013.0,1.0,Shawn Marion makes free throw 1 of 1,Shawn Marion,regular,21 - 12,2006 - 2007,1,7:18


In [3]:
from statsmodels.stats.weightstats import ztest

new_df = {
    "player": [],
    "regular_mean": [],
    "playoff_mean": [],
    "p_value": []
}

for player, group in data.groupby("player"):
    regular_shots = group[group["playoffs"] == "regular"]["shot_made"].values
    playoff_shots = group[group["playoffs"] == "playoffs"]["shot_made"].values
    
    if len(regular_shots) < 30 or len(playoff_shots) < 30:
        continue
        
    statistic, p_value = ztest(regular_shots, playoff_shots)

    new_df["player"].append(player)
    new_df["regular_mean"].append(np.mean(regular_shots))
    new_df["playoff_mean"].append(np.mean(playoff_shots))
    new_df["p_value"].append(p_value)
    
new_df = pd.DataFrame(new_df)

In [5]:
new_df.sample(10)

,player,regular_mean,playoff_mean,p_value
249,Timofey Mozgov,0.730114,0.763441,0.494635
144,Keyon Dooling,0.819383,0.806452,0.855139
223,Ronny Turiaf,0.638184,0.557692,0.244418
211,Rasheed Wallace,0.771107,0.801887,0.487934
118,Jerry Stackhouse,0.855372,0.887097,0.499810
264,Zach Randolph,0.768387,0.733333,0.162017
44,Chris Andersen,0.681905,0.688623,0.862543
226,Russell Westbrook,0.816679,0.845649,0.082660
25,Ben Wallace,0.403162,0.435897,0.575475
79,Ed Davis,0.562997,0.578947,0.846240


In [6]:
from statsmodels.stats.multitest import multipletests
multipletests(new_df.p_value, alpha=0.05, method='bonferroni')[0].sum()

1

In [7]:
multipletests(new_df.p_value, alpha=0.05, method='holm')[0].sum()

1